In [1]:
import pandas as pd
import numpy as np
import keras
import gensim
import pickle
import gc
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
%matplotlib inline

%load_ext autoreload
%autoreload 2
import helper

Using TensorFlow backend.


In [2]:
X_train = np.load("data/word_vectors/pubmed_stratify_no_wiki_X_train_seeds42.npy")
y_train = np.load("data/word_vectors/pubmed_stratify_no_wiki_y_train_seeds42.npy")

In [3]:
X_test = np.load("data/word_vectors/pubmed_stratify_no_wiki_X_test_seeds42.npy")
y_test = np.load("data/word_vectors/pubmed_stratify_no_wiki_y_test_seeds42.npy")

In [4]:
X_train = np.concatenate((X_train, X_test), axis=0)
y_train = np.concatenate((y_train, y_test), axis=0)

In [5]:
X_train.shape

(45587, 100, 200)

In [6]:
input_shape = (100, 200)

# Model

In [7]:
input_shape = (100, 200)
filters = 128
kernel_size = 1
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(filters,
             kernel_size,
             padding='valid',
             activation='relu',
             strides=1, input_shape=input_shape))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Conv1D(256,
             2,
             padding='valid',
             activation='relu',
             strides=1))
model.add(keras.layers.MaxPool1D(pool_size=32))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.recurrent.GRU(128))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(23, activation='softmax'))

In [8]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False) # Just a sanity check, obviously test was part of training
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 100, 128)          25728     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 99, 256)           65792     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 256)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 256)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               147840    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0

In [9]:
del X_train
del X_test

In [10]:
gc.collect()

0

# Submission

In [11]:
_,target, raw_cats = helper.load_multiclass_data()

In [12]:
submission_vectors = np.load("data/submission_word_vectors.npy")
submission_ids = np.load("data/submission_ids.npy")

In [13]:
cat_names = list(target.columns)
def one_hot_to_cats(one_hot_row):
    for index, col in enumerate(one_hot_row):
        if col == 1:
            return cat_names[index]

In [14]:
cat_names

['Animal Diseases',
 'Bacterial Infections and Mycoses',
 'Cardiovascular Diseases',
 'Digestive System Diseases',
 'Disorders of Environmental Origin',
 'Endocrine Diseases',
 'Eye Diseases',
 'Female Genital Diseases and Pregnancy Complications',
 'Hemic and Lymphatic Diseases',
 'Immunologic Diseases',
 'Musculoskeletal Diseases',
 'Neonatal Diseases and Abnormalities',
 'Neoplasms',
 'Nervous System Diseases',
 'Nutritional and Metabolic Diseases',
 'Otorhinolaryngologic Diseases',
 'Parasitic Diseases',
 'Pathological Conditions, Signs and Symptoms',
 'Respiratory Tract Diseases',
 'Skin and Connective Tissue Diseases',
 'Stomatognathic Diseases',
 'Urologic and Male Genital Diseases',
 'Virus Diseases']

### Reconstruct the categories from the one-hot encoding

In [15]:
yhat = model.predict(submission_vectors)

In [16]:
yhat_cats = np.apply_along_axis(helper.max_to_one, arr=yhat, axis=1)

In [17]:
submission_cats = list()
for row in yhat_cats:
    submission_cats.append(one_hot_to_cats(row))
submission_cats[0:10]

['Parasitic Diseases',
 'Eye Diseases',
 'Bacterial Infections and Mycoses',
 'Respiratory Tract Diseases',
 'Parasitic Diseases',
 'Cardiovascular Diseases',
 'Urologic and Male Genital Diseases',
 'Musculoskeletal Diseases',
 'Cardiovascular Diseases',
 'Skin and Connective Tissue Diseases']

In [18]:
test = pd.get_dummies(submission_cats)
for i in range(len(submission_vectors)):
    # if all elements of the arrays match we get 23 True values, which gives a sum of 23
    if np.sum(yhat_cats[i] == test.values[i]) != 23:
        print(f"Error at {i}")

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


Error at 0
Error at 1
Error at 2
Error at 3
Error at 4
Error at 5
Error at 6
Error at 7
Error at 8
Error at 9
Error at 10
Error at 11
Error at 12
Error at 13
Error at 14
Error at 15
Error at 16
Error at 17
Error at 18
Error at 19
Error at 20
Error at 21
Error at 22
Error at 23
Error at 24
Error at 25
Error at 26
Error at 27
Error at 28
Error at 29
Error at 30
Error at 31
Error at 32
Error at 33
Error at 34
Error at 35
Error at 36
Error at 37
Error at 38
Error at 39
Error at 40
Error at 41
Error at 42
Error at 43
Error at 44
Error at 45
Error at 46
Error at 47
Error at 48
Error at 49
Error at 50
Error at 51
Error at 52
Error at 53
Error at 54
Error at 55
Error at 56
Error at 57
Error at 58
Error at 59
Error at 60
Error at 61
Error at 62
Error at 63
Error at 64
Error at 65
Error at 66
Error at 67
Error at 68
Error at 69
Error at 70
Error at 71
Error at 72
Error at 73
Error at 74
Error at 75
Error at 76
Error at 77
Error at 78
Error at 79
Error at 80
Error at 81
Error at 82
Error at 83
Er

Error at 1822
Error at 1823
Error at 1824
Error at 1825
Error at 1826
Error at 1827
Error at 1828
Error at 1829
Error at 1830
Error at 1831
Error at 1832
Error at 1833
Error at 1834
Error at 1835
Error at 1836
Error at 1837
Error at 1838
Error at 1839
Error at 1840
Error at 1841
Error at 1842
Error at 1843
Error at 1844
Error at 1845
Error at 1846
Error at 1847
Error at 1848
Error at 1849
Error at 1850
Error at 1851
Error at 1852
Error at 1853
Error at 1854
Error at 1855
Error at 1856
Error at 1857
Error at 1858
Error at 1859
Error at 1860
Error at 1861
Error at 1862
Error at 1863
Error at 1864
Error at 1865
Error at 1866
Error at 1867
Error at 1868
Error at 1869
Error at 1870
Error at 1871
Error at 1872
Error at 1873
Error at 1874
Error at 1875
Error at 1876
Error at 1877
Error at 1878
Error at 1879
Error at 1880
Error at 1881
Error at 1882
Error at 1883
Error at 1884
Error at 1885
Error at 1886
Error at 1887
Error at 1888
Error at 1889
Error at 1890
Error at 1891
Error at 1892
Error 

Error at 4479
Error at 4480
Error at 4481
Error at 4482
Error at 4483
Error at 4484
Error at 4485
Error at 4486
Error at 4487
Error at 4488
Error at 4489
Error at 4490
Error at 4491
Error at 4492
Error at 4493
Error at 4494
Error at 4495
Error at 4496
Error at 4497
Error at 4498
Error at 4499
Error at 4500
Error at 4501
Error at 4502
Error at 4503
Error at 4504
Error at 4505
Error at 4506
Error at 4507
Error at 4508
Error at 4509
Error at 4510
Error at 4511
Error at 4512
Error at 4513
Error at 4514
Error at 4515
Error at 4516
Error at 4517
Error at 4518
Error at 4519
Error at 4520
Error at 4521
Error at 4522
Error at 4523
Error at 4524
Error at 4525
Error at 4526
Error at 4527
Error at 4528
Error at 4529
Error at 4530
Error at 4531
Error at 4532
Error at 4533
Error at 4534
Error at 4535
Error at 4536
Error at 4537
Error at 4538
Error at 4539
Error at 4540
Error at 4541
Error at 4542
Error at 4543
Error at 4544
Error at 4545
Error at 4546
Error at 4547
Error at 4548
Error at 4549
Error 

Error at 7490
Error at 7491
Error at 7492
Error at 7493
Error at 7494
Error at 7495
Error at 7496
Error at 7497
Error at 7498
Error at 7499
Error at 7500
Error at 7501
Error at 7502
Error at 7503
Error at 7504
Error at 7505
Error at 7506
Error at 7507
Error at 7508
Error at 7509
Error at 7510
Error at 7511
Error at 7512
Error at 7513
Error at 7514
Error at 7515
Error at 7516
Error at 7517
Error at 7518
Error at 7519
Error at 7520
Error at 7521
Error at 7522
Error at 7523
Error at 7524
Error at 7525
Error at 7526
Error at 7527
Error at 7528
Error at 7529
Error at 7530
Error at 7531
Error at 7532
Error at 7533
Error at 7534
Error at 7535
Error at 7536
Error at 7537
Error at 7538
Error at 7539
Error at 7540
Error at 7541
Error at 7542
Error at 7543
Error at 7544
Error at 7545
Error at 7546
Error at 7547
Error at 7548
Error at 7549
Error at 7550
Error at 7551
Error at 7552
Error at 7553
Error at 7554
Error at 7555
Error at 7556
Error at 7557
Error at 7558
Error at 7559
Error at 7560
Error 

Error at 10148
Error at 10149
Error at 10150
Error at 10151
Error at 10152
Error at 10153
Error at 10154
Error at 10155
Error at 10156
Error at 10157
Error at 10158
Error at 10159
Error at 10160
Error at 10161
Error at 10162
Error at 10163
Error at 10164
Error at 10165
Error at 10166
Error at 10167
Error at 10168
Error at 10169
Error at 10170
Error at 10171
Error at 10172
Error at 10173
Error at 10174
Error at 10175
Error at 10176
Error at 10177
Error at 10178
Error at 10179
Error at 10180
Error at 10181
Error at 10182
Error at 10183
Error at 10184
Error at 10185
Error at 10186
Error at 10187
Error at 10188
Error at 10189
Error at 10190
Error at 10191
Error at 10192
Error at 10193
Error at 10194
Error at 10195
Error at 10196
Error at 10197
Error at 10198
Error at 10199
Error at 10200
Error at 10201
Error at 10202
Error at 10203
Error at 10204
Error at 10205
Error at 10206
Error at 10207
Error at 10208
Error at 10209
Error at 10210
Error at 10211
Error at 10212
Error at 10213
Error at 1

In [19]:
submission_df = pd.DataFrame()
submission_df["Id"] = submission_ids
submission_df["Category"] = submission_cats

In [20]:
submission_df.head()

,Id,Category
0,4132,Parasitic Diseases
1,23198,Eye Diseases
2,8,Bacterial Infections and Mycoses
3,34357,Respiratory Tract Diseases
4,3800,Parasitic Diseases


In [21]:
submission_df.to_csv("data/multiclass_submissions_recurrent_cnn.csv", index=False)